OUTLINE

MODULE 1: Data preprocessing and loading

(a) Obtaining and cleaning the datasets

(b) Twitter specific text pre-processing

(c) Word / sentence vectorisation (as first data input)

Michael: BERT and FastText

(d) Implementing a dictionary approach potentially based on Hatebase.org (as second data input)
(e) Splitting data into test, validation and testing set
(f) Specifying and implementing the data loader
(g) Testing and iterating the module
(h) Creatingmodule-specificvisualisationsforthefinal paper

MODULE 2: Model architecture and training

(a) Choosing width and depth of the model
(b) Choosingoptimizeraswellasactivationandloss functions
(c) Choosing stopping rule, regularisation, dropout, learning rates etc.
(d) Potentially performing a hyperparameter grid search
(e) Running and tracking the training
(f) Training and tuning the model based on the re- sults of the validation set
(g) Creatingmodule-specificvisualisationsforthefi- nal paper

In [12]:
import pandas as pd

In [13]:
# (a) Obtaining and cleaning the datasets
url_davidson = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/davidson%20et%20al./labeled_data.csv"
url_founta = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/founta%20et%20al./hatespeech_text_label_vote.csv"


data_davidson = pd.read_csv(url_davidson, names=["count", "hate_speech", "offensive_language", "neither", "label", "tweet"], header=1)
data_founta = pd.read_csv(url_founta, sep='\t', names=["tweet", "label_text", "count"])

data_founta = data_founta[~data_founta.label_text.str.contains("spam")]
data_founta['label'] = data_founta.label_text.replace({'hateful': '0', 'abusive': '1', 'normal': '2'}).astype('int')
data_founta.label.hist()

# concatinating and shuffling


HTTPError: HTTP Error 404: Not Found

In [ ]:
data_davidson.label.hist()

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/4th Semester/NLP/nlp-project/coding/code/exchange_base/data.csv")
df.label.hist()

In [ ]:
# trying to re-import hatebase, somehow doesn't work

'''
import json 
import requests
import pandas as pd
from hatebase import HatebaseAPI

key = open('C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/Hatebase_Key', "r").read() # insert filepath where the API key is stored
filepath = "C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/full_dictionary.csv" # insert filepath where the final csv file should be stored 

hatebase = HatebaseAPI({"key": key})
filters = {"language": "eng"}
format = "json"
# initialize list for all vocabulary entry dictionaries
eng_vocab = []
response = hatebase.getVocabulary(filters=filters, format=format)
pages = response["number_of_pages"]
# fill the vocabulary list with all entries of all pages
# this might take some time...
for page in range(1, pages+1):
    filters["page"] = str(page) 
    response = hatebase.getVocabulary(filters=filters, format=format)
    eng_vocab.append(response["result"])

data = []
for r in eng_vocab:
    data.append(r["data"])
#print len(data)
listofHatewords = []

#print len(data)
for z in data:
    for a, v in z.iteritems():
        for b in v:
            listofHatewords.append(b["vocabulary"])
# print listofHatewords
listofHatewords = list(OrderedDict.fromkeys(listofHatewords))
print(len(listofHatewords))
'''


In [27]:
# Loading packages
#import csv
#import sys
#import nltk
import pandas as pd
import torch
import spacy
import dframcy
import pandarallel

#def dict_approach(data = None, lemmatization = True)
    """This function matches the terms in the Hatebase.org dictionary with the tweets in our dataset.
    First, both datasets are lemmatized, then the function counts the number of hatebase.org terms appearing in each tweet and adds the count as well as the average offensiveness (as defined by hatebase.org methodology). These two outputs are then added to the original dataset and further transformed into a tensor for further analysis.
    The input dataframe (tweets) can be specified with the variable "data". If nothing is specified, the function will get the data from our GitHub repository."""

# Loading the data

# loading Hatebase dictionary
hatebase_url = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/dictionary/hatebase/full_dictionary.csv"
hatebase_dic = pd.read_csv(hatebase_url, index_col = 'vocabulary_id')

# loading the tweets
data_url = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/data.csv"

# reading the data in case it is not here already (since it will later all be applied in one pipeline)
if data is None:
    data = pd.read_csv(data_url, index_col = 'id')
    print('INFO: Reading data anew from GitHub since no input was provided')


In [24]:
# Lemmatizing both the dictionary and the tweets
#if lemmatization:
# loading the spacy model
nlp = spacy.load('en_core_web_sm')
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# lemmatizing the dictionary
lemma = []
for term in nlp.pipe(hatebase_dic['term'].astype('unicode').values, batch_size=50, n_threads=3):
    if term.is_parsed:
        lemma.append([n.lemma_ for n in term])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
hatebase_dic['term_lemma'] = lemma

#making lemmas lowercase
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: " ".join(lemmas))

# lemmatizing the tweets
lemma = []
for tweet in nlp.pipe(data['tweet'].astype('unicode').values, batch_size=50, n_threads=3):
    if tweet.is_parsed:
        lemma.append([n.lemma_ for n in tweet])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
data['tweet_lemma'] = lemma

#making lemmas lowercase
data['tweet_lemma'] = data['tweet_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
data['tweet_lemma'] = data['tweet_lemma'].map(lambda lemmas: " ".join(lemmas))

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [22]:
# Searching for HateBase words in tweets

pandarallel.initialize() 

# defining the search function
def hatesearch(data):
    frequency = 0
    hatefulness = 0
    for term_lemma in hatebase_dic:
        if term_lemma in data['tweet_lemma']:
            frequency += 1
            hatefulness = hatebase_dic[hatebase_dic['term_lemma'] == term_lemma].average_offensiveness
    data['Hatefreq'] = frequency
    data['Hatefulness'] = hatefulness
    return data

# running the search function
data = data.parallel_apply(hatesearch, axis = 1)

#data = data.apply(lambda x : hatesearch, axis=1)

#print(data)

# Exporting to browse through

#data.to_csv('C:/Users/Niko/Documents/newdata.csv')

AttributeError: module 'pandarallel' has no attribute 'initialize'

In [21]:
# Building a tensor out of the additional columns

HateFrequency = torch.tensor(data['Hatefreq'])
torch.save(HateFrequency, 'https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/HateFreq_Tensor.pt')

HateIntensity = torch.tensor(data['Hatefulness'])
torch.save(HateFrequency, 'https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/HateIntens_Tensor.pt')

KeyError: 'Hatefreq'

In [51]:
### Trying to 

# loading packages
import pandas as pd
import numpy as np
import transformers
import torch
import tqdm
import spacy
from pandarallel import pandarallel
import os

# Loading the dictionary

hatebase_path = '../exchange_base/full_dictionary.csv'
hatebase_dic = pd.read_csv(hatebase_path, index_col = 'vocabulary_id')

# loading the spacy model
nlp = spacy.load('en_core_web_sm')
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# lemmatizing the dictionary
lemma = []
for term in nlp.pipe(hatebase_dic['term'].astype('unicode').values, batch_size=50, n_threads=3):
    if term.is_parsed:
        lemma.append([n.lemma_ for n in term])
    else:
        lemma.append(None)

# storing the lemmas in a new column 
hatebase_dic['term_lemma'] = lemma

#making lemmas lowercase
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
#joining list of lemmas into one string again, so it becomes matchable by the searchfunction
hatebase_dic['term_lemma'] = hatebase_dic['term_lemma'].map(lambda lemmas: " ".join(lemmas))

hatebase_dic = hatebase_dic.drop_duplicates(subset=['term_lemma'])

# Loading the tensors

train_vectorized = torch.load('../exchange_base/train_vectorized_1d.pt')
print(train_vectorized)
print(train_vectorized[0])
print(train_vectorized[0][0])

# Loading BERT

pretrainedModel="bert-base-uncased"
tokenizer = transformers.BertTokenizer.from_pretrained(pretrainedModel)


train_vectorized_tweet1 = train_vectorized[0]

# Decoding

tweet1 = tokenizer.decode(train_vectorized_tweet, skip_special_tokens=True, clean_up_tokenization_spaces = True)

print(type(tweet1))
print(tweet1)


#tweets = tokenizer.decode(train_vectorized, skip_special_tokens=True, clean_up_tokenization_spaces = True) ### does not work!

df = pd.DataFrame(data=tweet1, dtype=str)
print(df)

'''
def hatesearch(column):
    frequency = 0
    hatefulness_sum = 0
    for hateterm in hatebase_dic['term_lemma']:
        if hateterm in column['text']:
            frequency += 1
            row_hatebase_dict_term_lemma = hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'average_offensiveness'].iloc[0]
            if np.isnan(row_hatebase_dict_term_lemma):
                hatefulness_term = 77.27734806629834
            else: 
                hatefulness_term = row_hatebase_dict_term_lemma
            if hatebase_dic.loc[hatebase_dic['term_lemma'] == hateterm, 'is_unambiguous'].iloc[0] == False:
                hatefulness_term_weighted = hatefulness_term
            else:
                hatefulness_term_weighted = hatefulness_term*2
            hatefulness_sum += hatefulness_term_weighted
    column['Hatefreq'] = frequency
    if frequency != 0:
        row['Hatefulness'] = (hatefulness_sum / frequency)
    else:
        row['Hatefulness'] = 0
    return row
'''

tensor([[  101,  1030, 19031,  ...,     0,     0,     0],
        [  101,  5187,  2487,  ...,     0,     0,     0],
        [  101,  6842,  2380,  ...,     0,     0,     0],
        ...,
        [  101,  2217,  7743,  ...,     0,     0,     0],
        [  101, 19522,  2632,  ...,     0,     0,     0],
        [  101,  2179,  1037,  ...,     0,     0,     0]], dtype=torch.int32)
tensor([  101,  1030, 19031, 10772,  6843,  2594,  2017,  2228,  1996,  2293,
         6771,  2005,  7055,  2097,  2058, 15637,  2008,  2002,  1005,  1055,
         3920,  1998,  2062,  3144,  1029,  1029,  1029,  2054,  2515,  2008,
         2812,  1029,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,   

ValueError: DataFrame constructor not properly called!